In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!huggingface-cli whoami

joshl1
orgs:  UMBC


## Installations
 Install the necessary libraries.

In [3]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.8 MB/s eta 0:00:00


## Setup the model

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-2-7b-chat-hf"

This next step will take a couple minutes to download the model.

In [5]:
# From the Llama-2 Chat Tutorial
# https://colab.research.google.com/drive/1X1z9Q6domMKl2CnEM0QGHNwidLfR4dW2?usp=sharing#scrollTo=THqfvzHIjSK9

quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [15]:
def generate_game_response(current_situation, action):
    prompt = f"""You are narrating an interactive fiction game. Given the current situation and player's action, describe what happens next in 2-3 sentences. Focus only on the immediate new situation.

Current situation: {current_situation}
Player action: {action}

What happens next:"""

    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(**model_inputs, max_length=1024, temperature=0.7)  # Reduced max_length for shorter response
    new_situation = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()

    # Only use the new situation as the current situation for the next turn
    return new_situation

In [16]:
# Generate initial game setting
setting_prompt = """You are creating an interactive fiction game. Describe the initial setting and situation the player finds themselves in. Keep it to 2-3 sentences."""

model_inputs = tokenizer(setting_prompt, return_tensors="pt").to("cuda:0")
output = model.generate(**model_inputs, max_length=1024, temperature=0.7)
current_situation = tokenizer.decode(output[0], skip_special_tokens=True)

print("\nWelcome to the AI Generated Adventure!")
print("\nYour situation:", current_situation)

# Main game loop
while True:
    print("\nWhat would you like to do? (type 'quit' to end)")
    action = input("> ")

    if action.lower() == 'quit':
        print("\nThanks for playing!")
        break

    response = generate_game_response(current_situation, action)
    current_situation = response
    print("\n" + response)


Welcome to the AI Generated Adventure!

Your situation: You are creating an interactive fiction game. Describe the initial setting and situation the player finds themselves in. Keep it to 2-3 sentences.

The player finds themselves in a dark forest, surrounded by towering trees that block out most of the sunlight. The air is thick with the scent of damp earth and decaying leaves, and the only sound is the distant howling of wolves. As they look around, they notice a small clearing with a glowing path of light leading through the trees.

What would you like to do? (type 'quit' to end)
> I fight the wolves with my sword.

As you charge towards the wolves with your sword raised, they turn and run at you, their eyes glowing in the dark. You swing your sword and manage to hit one of them, but there are too many of them and they overwhelm you, their jaws closing in on you.

What would you like to do? (type 'quit' to end)
> I use the sword to defend myself.

The wolves continue to attack you